In [1]:
%pip install pytorch_tabnet
%pip install catboost
%pip install https://github.com/schufa-innovationlab/model-trees/archive/master.zip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 68.2 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.0.0+cu118
    Uninstalling torch-2.0.0+cu118:
      Successfully uninstalled torch-2.0.0+cu118
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.0.1+cu118 requires torch==2.0.0, but you have torch 1.13.1 whi

In [2]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from keras.regularizers import l1, l2, l1_l2
from keras.optimizers import Adam

from pytorch_tabnet.tab_model import TabNetClassifier, TabNetRegressor

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, make_scorer
import statsmodels.api as sm
from statsmodels.formula.api import ols
import statsmodels.stats.api as sms
from statsmodels.compat import lzip
from statsmodels.stats.outliers_influence import OLSInfluence
import matplotlib.pyplot as plt
import math
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import lightgbm as lgb
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from catboost import CatBoostClassifier, CatBoostRegressor
from modeltrees import ModelTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.utils import shuffle
from sklearn.decomposition import PCA

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/MyDrive/244

Mounted at /content/gdrive
/content/gdrive/MyDrive/244


In [5]:
def hardy_cv(name, model, dataset, cv=5):  # We'll train only
  cv_scores = cross_val_score(
      model, X_train, Y_train, cv=cv, scoring='accuracy')
  print('accuracy = ', cv_scores.mean())
  return { 'mean': cv_scores.mean(), 'scores': cv_scores }

def hardy_train_with_validation(name, model, dataset):  # We'll train only
  X_train = dataset['X_train']
  Y_train = dataset['Y_train']

  X_train_train, X_train_val, Y_train_train, Y_train_val = train_test_split(
    X_train, Y_train, test_size=0.2, random_state=100)
  eval_set = [(X_train_val, Y_train_val)]
  model.fit(X_train_train, Y_train_train,
            eval_set = eval_set, metrics='accuracy',
            verbose=False, early_stopping_rounds=10)

def hardy_train(name, model, dataset):  # We'll train only
  X_train = dataset['X_train']
  Y_train = dataset['Y_train']
  model.fit(X_train, Y_train)

def hardy_predict(name, model, dataset):  # we'll predict only
  global all_results
  X_train = dataset['X_train']
  Y_train = dataset['Y_train']
  X_test = dataset['X_test']
  Y_test = dataset['Y_test']
  Y_train_pred = model.predict(X_train)
  Y_test_pred = model.predict(X_test)
  if name == 'Model Tree' or name == "Gaussian Process":
    Y_test_proba = None
  else:
    Y_test_proba = model.predict_proba(X_test)

  metrics = hardy_evaluate(name + " (train)", Y_train, Y_train_pred)
  metrics = hardy_evaluate(name, Y_test, Y_test_pred)
  # hardy_roc_auc(name, Y_test, Y_test_proba)
  results = { 'metrics': metrics, 'pred': Y_test_pred, 'proba': Y_test_proba }
  all_results[name] = results # posting the result to all_results{}
  return results

def hardy_predict_nn(name, model, dataset):  # we'll predict only
  global all_results  
  X_test = dataset['X_test']
  Y_test = dataset['Y_test']
  Y_proba = model.predict(X_test)
  Y_pred = np.argmax(Y_proba, axis=1)
  metrics = hardy_evaluate(name, Y_test, Y_pred)
  results = { 'metrics': metrics, 'pred': Y_pred, 'proba': Y_proba }
  all_results[name] = results # posting the result to all_results{}
  return results

def hardy_train_predict(name, model, dataset):  # train and predict
  hardy_train(name, model, dataset)
  return hardy_predict(name, model, dataset)

def hardy_roc_auc(name, Y_test, Y_test_proba):
  Y_test_proba_1d = [v[1] for v in Y_test_proba]
  roc_auc = roc_auc_score(Y_test, Y_test_proba_1d)
  fpr, tpr, thresholds = roc_curve(Y_test, Y_test_proba_1d)
  plt.plot(fpr, tpr, label='ROC curve (AUC = %0.2f)' % roc_auc)
  plt.plot([0, 1], [0, 1], 'k--')  # Diagonal line representing random guessing
  plt.xlabel('False Positive Rate')
  plt.ylabel('True Positive Rate')
  plt.title('Receiver Operating Characteristic')
  plt.legend(loc='lower right')
  plt.show()

def hardy_evaluate(name, Y_test, Y_pred):  # evaluate only
  accuracy = accuracy_score(Y_test, Y_pred)
  precision = precision_score(Y_test, Y_pred)
  recall = recall_score(Y_test, Y_pred)
  f1 = f1_score(Y_test, Y_pred)
  print(f'"{name}": accuracy = {accuracy}')
  print(f'"{name}": precision = {precision}')
  print(f'"{name}": recall = {recall}')
  print(f'"{name}": f1 = {f1}')
  cm = confusion_matrix(Y_test, Y_pred)
  print('-------------------------------------')
  print("Confusion Matrix:")
  print('-------------------------------------')
  print('               L      W (predicted)')
  for i in range(2):  # binary
    row_str = "    ".join([str(count) for count in cm[i]])
    truth = 'L (truth)' if i == 0 else 'W (truth)'
    print(f"  {truth}: {row_str}")
  print('-------------------------------------')
  return {
      'accuracy': accuracy,
      'precision': precision,
      'recall': recall,
      'f1': f1,
  }

def hardy_experiment(name, model, dataset):
  print('Experiment: train and predict ...')
  hardy_train_predict(name, model, dataset)
  
def hardy_pca(dataset, N=20):
  pca = PCA(n_components=N)

  X_train_pca = pd.DataFrame(pca.fit_transform(dataset['X_train']))
  X_test_pca = pd.DataFrame(pca.transform(dataset['X_test']))
  return {
      'X_train': X_train_pca,
      'Y_train': dataset['Y_train'],
      'X_test': X_test_pca,
      'Y_test': dataset['Y_test'],
  }

In [6]:
def run_lr():
  lr_model = LogisticRegression()
  hardy_experiment("Linear Regression", lr_model, dataset)

def run_xgb():
  xgb_model = xgb.XGBClassifier(objective= 'binary:logistic')
  hardy_experiment('XGBoost', xgb_model, dataset)

def run_lgb():
  lgb_model = lgb.LGBMClassifier()
  hardy_experiment("LightGBM", lgb_model, dataset)

def run_knn():  # crash
  knn_model = KNeighborsClassifier()  # n_neighbors=5
  hardy_experiment("KNN", knn_model, dataset)

def run_gaussian_process():
  gp_kernel = RBF(length_scale=1.0)
  gp_model = GaussianProcessClassifier(kernel=gp_kernel)
  hardy_experiment("Gaussian Process", gp_model, dataset)

def run_svn():
  svc_model = SVC(kernel='rbf', probability=True)
  hardy_experiment("SVM Classifier", svc_model, dataset)

def run_naive_bayes():
  naivebayes_model = GaussianNB() # don't do grid-search
  hardy_experiment('Naive Bayes', naivebayes_model, dataset)

def run_decision_tree():
  decision_tree_model = DecisionTreeClassifier()
  hardy_experiment("Decision Tree", decision_tree_model, dataset)
  
def run_adaboost():
  adaboost_model = AdaBoostClassifier()
  hardy_experiment("AdaBoost", adaboost_model, dataset)

def run_random_forest():
  random_forest_model = RandomForestClassifier()
  hardy_experiment("Random Forest", random_forest_model, dataset)
  
def run_qda():
  qda_model = QuadraticDiscriminantAnalysis()
  hardy_experiment("Quadratic Discriminant Analysis",
                                 qda_model, dataset)
  
def run_catboost():
  catboost_model = CatBoostClassifier(logging_level='silent')
  hardy_experiment("CatBoost", catboost_model, dataset)

def run_model_tree():
  dataset_numpy = {
      'X_train': dataset['X_train'].to_numpy(),
      'Y_train': dataset['Y_train'].to_numpy(),
      'X_test': dataset['X_test'].to_numpy(),
      'Y_test': dataset['Y_test'].to_numpy(),
  }
  mt_model = ModelTreeClassifier()
  hardy_experiment("Model Tree", mt_model, dataset_numpy)

def run_ensemble(names, algo=None):
  if algo == None:
    run_ensemble(names, 'add')
    run_ensemble(names, 'mult')
    run_ensemble(names, 'vote')
    return
  
  assert(algo == 'add' or algo == 'mult' or algo == 'vote')
  predictions = [all_results[name]['proba'] for name in names]
  Y_test = dataset['Y_test']
  Y_pred = [1] * len(Y_test)
  for i in range(len(Y_test)):
    prob_L = 0
    prob_W = 0
    for p in predictions:
      if algo == 'add':
        prob_L += p[i][0]
        prob_W += p[i][1]
      elif algo == 'mult':  # mult = add in log-scale
        prob_L += math.log(p[i][0] + 1e-9)
        prob_W += math.log(p[i][1] + 1e-9)
      elif algo == 'vote':
        prob_L += p[i][0] >= 0.5
        prob_W += p[i][1] >= 0.5
      else:
        assert(False)
    Y_pred[i] = 1 if prob_W >= prob_L else 0
  metrics = hardy_evaluate(f'Ensemble/{algo}', Y_test, Y_pred)
  all_results[f'Ensemble/{algo}'] = {
    'metrics': metrics
  }

def run_elo(feature='last'):
  if GLOBAL_PCA:
    return # cannot run this when we have done PCA

  Y_test = dataset['Y_test']
  Y_pred = []
  for i, r in X_test.iterrows():
    Y_pred.append(
        1 if r[f'elo-home-{feature}'] >= r[f'elo-away-{feature}'] else 0)
  metrics = hardy_evaluate(f'Elo ({feature})', Y_test, Y_pred)
  all_results[f'Elo ({feature})'] = {
    'metrics': metrics
  }


In [7]:
def run_nn():

  X_train = dataset['X_train']
  Y_train = dataset['Y_train']
  X_train_train, X_train_val, Y_train_train, Y_train_val = train_test_split(
    X_train, Y_train, test_size=0.2, random_state=100)

  label_encoder = LabelEncoder()
  Y_train_train_encoded = label_encoder.fit_transform(Y_train_train)
  Y_train_val_encoded = label_encoder.fit_transform(Y_train_val)

  # Convert to one-hot encoded vectors
  Y_train_train_one_hot = np_utils.to_categorical(Y_train_train_encoded)
  Y_train_val_one_hot = np_utils.to_categorical(Y_train_val_encoded)

  #########################################################################

  l2_value = 0.01
  nn_model = Sequential()
  nn_model.add(Dense(100, input_dim=X_train_train.shape[1], activation='relu',
                    kernel_regularizer=l2(l2_value)))
  nn_model.add(Dropout(.2))
  nn_model.add(Dense(100, activation='relu',
              kernel_regularizer=l2(l2_value)))
  nn_model.add(Dropout(.2))
  nn_model.add(Dense(100, activation='relu',
              kernel_regularizer=l2(l2_value)))
  nn_model.add(Dropout(.2))
  nn_model.add(Dense(2, activation='softmax'))

  #########################################################################

  optimizer = Adam(learning_rate=0.001)
  nn_model.compile(
      loss='categorical_crossentropy',
      optimizer=optimizer, metrics=['accuracy'])
  nn_model.fit(X_train_train, Y_train_train_one_hot,
              validation_data=(X_train_val, Y_train_val_one_hot),
              epochs=15, batch_size=32)
  
  nn_result = hardy_predict_nn('Neural Network', nn_model, dataset)

In [8]:
def run_tabnet():
  global all_results
  X_train = dataset['X_train']
  Y_train = dataset['Y_train']
  X_test = dataset['X_test']
  Y_test = dataset['Y_test']
  X_train_train, X_train_val, Y_train_train, Y_train_val = train_test_split(
    X_train, Y_train, test_size=0.2, random_state=100)

  X_train_train = X_train_train.to_numpy()
  X_train_val = X_train_val.to_numpy()
  Y_train_train = Y_train_train.to_numpy().squeeze()
  Y_train_val = Y_train_val.to_numpy().squeeze()

  print(X_train_train.shape)
  print(X_train_val.shape)
  print(Y_train_train.shape)
  print(Y_train_val.shape)

  tabnet_model = TabNetClassifier()
  tabnet_model.fit(
    X_train_train, Y_train_train,
    eval_set=[(X_train_train, Y_train_train), (X_train_val, Y_train_val)],
    max_epochs=100, patience=10)
  
  X_test = dataset['X_test']
  Y_test = dataset['Y_test']
  X_test = X_test.to_numpy()
  Y_test = Y_test.to_numpy().squeeze()

  Y_pred = tabnet_model.predict(X_test)
  Y_proba = tabnet_model.predict_proba(X_test)
  metrics = hardy_evaluate('TabNet', dataset['Y_test'], Y_pred)

  results = {
    'metrics': metrics,
    'pred': Y_pred,
    'proba': Y_proba }
  all_results['TabNet'] = results # posting the result to all_results{}
  return results

# Specification

In [9]:
TABULAR_CSV = './tabular-100-100-original.csv'
GLOBAL_PCA = False

In [10]:
tabular = pd.read_csv(TABULAR_CSV)
  # see report for which one to use

In [11]:
tabular = tabular.drop(['Unnamed: 0'], axis=1)
XY_train = tabular[tabular['date'] < '2019-06-30']
XY_test = tabular[tabular['date'] >= '2019-06-30']
XY_train = shuffle(XY_train, random_state=42)

print(f'train = {XY_train.shape}')
print(f'test = {XY_test.shape}')

X_train = XY_train.drop(['date', 'home', 'away', 'rivalry', 'label'], axis=1)
Y_train = XY_train['label']
X_test = XY_test.drop(['date', 'home', 'away', 'rivalry', 'label'], axis=1)
Y_test = XY_test['label']
print(f'X_train: {X_train.columns}')
dataset = {
    'X_train': X_train,
    'Y_train': Y_train,
    'X_test': X_test,
    'Y_test': Y_test,    
}
if GLOBAL_PCA:
  print('performing PCA ...')
  dataset = hardy_pca(dataset)
  X_train = dataset['X_train']
  Y_train = dataset['Y_train']
  X_test = dataset['X_test']
  Y_test = dataset['Y_test']

all_results = {}

train = (3468, 203)
test = (1105, 203)
X_train: Index(['passing-yard-ratio-home-last', 'fourth-down-conversion-home-last',
       'sack_count-home-last', 'third-downs-home-last', 'comp-att-home-last',
       'score-home-last', 'def-st-td-home-last', 'penalty_yards-home-last',
       'espn-qbr-home-last', 'fourth-downs-home-last',
       ...
       'int-away-last-season', 'redzone_count-away-last-season',
       'third-down-conversion-away-last-season', 'wintieloss-away-last-season',
       'redzone_yards-away-last-season', 'rushing-to-passing-away-last-season',
       'qb-elo-away-last-season', 'elo-away-last-season',
       'rushing-attempts-away-last-season', 'turnovers-away-last-season'],
      dtype='object', length=198)


In [12]:
X_train[-5:]

,passing-yard-ratio-home-last,fourth-down-conversion-home-last,sack_count-home-last,third-downs-home-last,comp-att-home-last,score-home-last,def-st-td-home-last,penalty_yards-home-last,espn-qbr-home-last,fourth-downs-home-last,...,int-away-last-season,redzone_count-away-last-season,third-down-conversion-away-last-season,wintieloss-away-last-season,redzone_yards-away-last-season,rushing-to-passing-away-last-season,qb-elo-away-last-season,elo-away-last-season,rushing-attempts-away-last-season,turnovers-away-last-season
1095,0.340909,0.0,2.0,11.0,0.476190,9.0,0.0,125.0,10.1,1.0,...,0.897785,2.093886,0.426515,0.601363,4.105811,0.462931,1638.082506,1630.509871,28.464698,1.452773
1130,0.781250,0.0,1.0,11.0,0.648649,17.0,0.0,52.0,78.2,1.0,...,0.759649,1.522655,0.354709,0.535521,3.263845,0.570606,1516.049052,1518.884043,27.653366,1.399217
1294,0.526448,0.0,0.0,13.0,0.608696,19.0,0.0,55.0,55.3,1.0,...,0.693485,2.087823,0.430429,0.756464,3.898743,0.358900,1592.089756,1601.272505,26.035534,1.124224
860,0.833333,0.5,2.0,10.0,0.658537,14.0,0.0,97.0,65.6,2.0,...,0.833705,1.319770,0.382791,0.580082,3.570977,0.628958,1534.047355,1526.884859,28.130806,1.637495
3174,0.797583,0.5,5.0,13.0,0.611111,13.0,0.0,91.0,56.5,2.0,...,1.192669,1.856504,0.366459,0.370586,3.074356,0.515197,1550.516945,1568.048144,28.296528,1.844286


In [13]:
print(list(X_train.columns))

['passing-yard-ratio-home-last', 'fourth-down-conversion-home-last', 'sack_count-home-last', 'third-downs-home-last', 'comp-att-home-last', 'score-home-last', 'def-st-td-home-last', 'penalty_yards-home-last', 'espn-qbr-home-last', 'fourth-downs-home-last', 'sack_yards-home-last', 'espn-qbr-missing-home-last', 'comp-home-last', 'rushing-yard-ratio-home-last', 'first-downs-home-last', 'att-home-last', 'total-yards-home-last', 'rushing-yards-home-last', 'passing-yards-home-last', 'possession-home-last', 'drives-home-last', 'penalty_count-home-last', 'fumbles-home-last', 'int-home-last', 'redzone_count-home-last', 'third-down-conversion-home-last', 'wintieloss-home-last', 'redzone_yards-home-last', 'rushing-to-passing-home-last', 'qb-elo-home-last', 'elo-home-last', 'rushing-attempts-home-last', 'turnovers-home-last', 'passing-yard-ratio-away-last', 'fourth-down-conversion-away-last', 'sack_count-away-last', 'third-downs-away-last', 'comp-att-away-last', 'score-away-last', 'def-st-td-away-

In [14]:
X_train

,passing-yard-ratio-home-last,fourth-down-conversion-home-last,sack_count-home-last,third-downs-home-last,comp-att-home-last,score-home-last,def-st-td-home-last,penalty_yards-home-last,espn-qbr-home-last,fourth-downs-home-last,...,int-away-last-season,redzone_count-away-last-season,third-down-conversion-away-last-season,wintieloss-away-last-season,redzone_yards-away-last-season,rushing-to-passing-away-last-season,qb-elo-away-last-season,elo-away-last-season,rushing-attempts-away-last-season,turnovers-away-last-season
1602,0.700000,0.0,2.0,14.0,0.390244,9.0,0.0,40.0,35.9,0.0,...,1.576774,2.047183,0.413667,0.478544,4.245272,0.649941,1548.540122,1531.303823,28.353264,2.388245
805,0.379939,0.0,1.0,16.0,0.600000,16.0,0.0,43.0,49.8,1.0,...,1.109853,1.973580,0.473099,0.519070,3.625149,0.449711,1461.577826,1477.574285,24.360151,1.882400
864,0.776699,0.5,6.0,11.0,0.538462,10.0,0.0,19.0,2.7,2.0,...,1.208608,0.917766,0.332858,0.259890,2.447436,1.028902,1452.836437,1468.493557,25.521062,1.578388
321,0.909722,0.0,10.0,14.0,0.630435,21.0,0.0,65.0,25.4,2.0,...,1.293814,1.569618,0.359685,0.808571,3.191763,0.778345,1617.987241,1615.845337,31.723362,2.212106
2119,0.559767,1.0,1.0,10.0,0.586207,38.0,2.0,58.0,64.7,2.0,...,1.153938,0.000000,0.304475,0.303715,0.000000,0.533694,1376.829643,1386.421213,24.573105,1.635958
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1095,0.340909,0.0,2.0,11.0,0.476190,9.0,0.0,125.0,10.1,1.0,...,0.897785,2.093886,0.426515,0.601363,4.105811,0.462931,1638.082506,1630.509871,28.464698,1.452773
1130,0.781250,0.0,1.0,11.0,0.648649,17.0,0.0,52.0,78.2,1.0,...,0.759649,1.522655,0.354709,0.535521,3.263845,0.570606,1516.049052,1518.884043,27.653366,1.399217
1294,0.526448,0.0,0.0,13.0,0.608696,19.0,0.0,55.0,55.3,1.0,...,0.693485,2.087823,0.430429,0.756464,3.898743,0.358900,1592.089756,1601.272505,26.035534,1.124224
860,0.833333,0.5,2.0,10.0,0.658537,14.0,0.0,97.0,65.6,2.0,...,0.833705,1.319770,0.382791,0.580082,3.570977,0.628958,1534.047355,1526.884859,28.130806,1.637495


In [15]:
run_elo()

"Elo (last)": accuracy = 0.6470588235294118
"Elo (last)": precision = 0.6695652173913044
"Elo (last)": recall = 0.6581196581196581
"Elo (last)": f1 = 0.6637931034482758
-------------------------------------
Confusion Matrix:
-------------------------------------
               L      W (predicted)
  L (truth): 330    190
  W (truth): 200    385
-------------------------------------


In [16]:
run_model_tree()

Experiment: train and predict ...
"Model Tree (train)": accuracy = 0.68800461361015
"Model Tree (train)": precision = 0.7046490428441203
"Model Tree (train)": recall = 0.7808080808080808
"Model Tree (train)": f1 = 0.740776233828462
-------------------------------------
Confusion Matrix:
-------------------------------------
               L      W (predicted)
  L (truth): 840    648
  W (truth): 434    1546
-------------------------------------
"Model Tree": accuracy = 0.616289592760181
"Model Tree": precision = 0.6148359486447932
"Model Tree": recall = 0.7367521367521368
"Model Tree": f1 = 0.6702954898911353
-------------------------------------
Confusion Matrix:
-------------------------------------
               L      W (predicted)
  L (truth): 250    270
  W (truth): 154    431
-------------------------------------


In [17]:
run_lr()

Experiment: train and predict ...
"Linear Regression (train)": accuracy = 0.6473471741637832
"Linear Regression (train)": precision = 0.6652116979484941
"Linear Regression (train)": recall = 0.7696969696969697
"Linear Regression (train)": f1 = 0.7136501990166236
-------------------------------------
Confusion Matrix:
-------------------------------------
               L      W (predicted)
  L (truth): 721    767
  W (truth): 456    1524
-------------------------------------
"Linear Regression": accuracy = 0.632579185520362
"Linear Regression": precision = 0.6211096075778079
"Linear Regression": recall = 0.7846153846153846
"Linear Regression": f1 = 0.6933534743202417
-------------------------------------
Confusion Matrix:
-------------------------------------
               L      W (predicted)
  L (truth): 240    280
  W (truth): 126    459
-------------------------------------


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [18]:
run_xgb()

Experiment: train and predict ...
"XGBoost (train)": accuracy = 0.998558246828143
"XGBoost (train)": precision = 1.0
"XGBoost (train)": recall = 0.9974747474747475
"XGBoost (train)": f1 = 0.9987357774968395
-------------------------------------
Confusion Matrix:
-------------------------------------
               L      W (predicted)
  L (truth): 1488    0
  W (truth): 5    1975
-------------------------------------
"XGBoost": accuracy = 0.5864253393665159
"XGBoost": precision = 0.5924855491329479
"XGBoost": recall = 0.7008547008547008
"XGBoost": f1 = 0.6421299921691463
-------------------------------------
Confusion Matrix:
-------------------------------------
               L      W (predicted)
  L (truth): 238    282
  W (truth): 175    410
-------------------------------------


In [19]:
run_lgb()

Experiment: train and predict ...
"LightGBM (train)": accuracy = 0.9968281430219147
"LightGBM (train)": precision = 0.9969712266532055
"LightGBM (train)": recall = 0.9974747474747475
"LightGBM (train)": f1 = 0.9972229235041656
-------------------------------------
Confusion Matrix:
-------------------------------------
               L      W (predicted)
  L (truth): 1482    6
  W (truth): 5    1975
-------------------------------------
"LightGBM": accuracy = 0.604524886877828
"LightGBM": precision = 0.6033519553072626
"LightGBM": recall = 0.7384615384615385
"LightGBM": f1 = 0.6641045349730976
-------------------------------------
Confusion Matrix:
-------------------------------------
               L      W (predicted)
  L (truth): 236    284
  W (truth): 153    432
-------------------------------------


In [20]:
run_svn()

Experiment: train and predict ...
"SVM Classifier (train)": accuracy = 0.6375432525951558
"SVM Classifier (train)": precision = 0.6321755027422303
"SVM Classifier (train)": recall = 0.8732323232323232
"SVM Classifier (train)": f1 = 0.7334040296924708
-------------------------------------
Confusion Matrix:
-------------------------------------
               L      W (predicted)
  L (truth): 482    1006
  W (truth): 251    1729
-------------------------------------
"SVM Classifier": accuracy = 0.6108597285067874
"SVM Classifier": precision = 0.5877689694224235
"SVM Classifier": recall = 0.8871794871794871
"SVM Classifier": f1 = 0.7070844686648501
-------------------------------------
Confusion Matrix:
-------------------------------------
               L      W (predicted)
  L (truth): 156    364
  W (truth): 66    519
-------------------------------------


In [21]:
run_naive_bayes()

Experiment: train and predict ...
"Naive Bayes (train)": accuracy = 0.6239907727797002
"Naive Bayes (train)": precision = 0.6443210930828351
"Naive Bayes (train)": recall = 0.7621212121212121
"Naive Bayes (train)": f1 = 0.6982878297084683
-------------------------------------
Confusion Matrix:
-------------------------------------
               L      W (predicted)
  L (truth): 655    833
  W (truth): 471    1509
-------------------------------------
"Naive Bayes": accuracy = 0.5972850678733032
"Naive Bayes": precision = 0.5875
"Naive Bayes": recall = 0.8034188034188035
"Naive Bayes": f1 = 0.6787003610108303
-------------------------------------
Confusion Matrix:
-------------------------------------
               L      W (predicted)
  L (truth): 190    330
  W (truth): 115    470
-------------------------------------


In [22]:
run_gaussian_process()

Experiment: train and predict ...
"Gaussian Process (train)": accuracy = 0.998558246828143
"Gaussian Process (train)": precision = 1.0
"Gaussian Process (train)": recall = 0.9974747474747475
"Gaussian Process (train)": f1 = 0.9987357774968395
-------------------------------------
Confusion Matrix:
-------------------------------------
               L      W (predicted)
  L (truth): 1488    0
  W (truth): 5    1975
-------------------------------------
"Gaussian Process": accuracy = 0.47058823529411764
"Gaussian Process": precision = 0.0
"Gaussian Process": recall = 0.0
"Gaussian Process": f1 = 0.0
-------------------------------------
Confusion Matrix:
-------------------------------------
               L      W (predicted)
  L (truth): 520    0
  W (truth): 585    0
-------------------------------------


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [23]:
run_decision_tree()

Experiment: train and predict ...
"Decision Tree (train)": accuracy = 0.998558246828143
"Decision Tree (train)": precision = 1.0
"Decision Tree (train)": recall = 0.9974747474747475
"Decision Tree (train)": f1 = 0.9987357774968395
-------------------------------------
Confusion Matrix:
-------------------------------------
               L      W (predicted)
  L (truth): 1488    0
  W (truth): 5    1975
-------------------------------------
"Decision Tree": accuracy = 0.5828054298642534
"Decision Tree": precision = 0.6
"Decision Tree": recall = 0.6358974358974359
"Decision Tree": f1 = 0.6174273858921161
-------------------------------------
Confusion Matrix:
-------------------------------------
               L      W (predicted)
  L (truth): 272    248
  W (truth): 213    372
-------------------------------------


In [24]:
run_adaboost()

Experiment: train and predict ...
"AdaBoost (train)": accuracy = 0.6972318339100346
"AdaBoost (train)": precision = 0.7123287671232876
"AdaBoost (train)": recall = 0.7878787878787878
"AdaBoost (train)": f1 = 0.7482014388489209
-------------------------------------
Confusion Matrix:
-------------------------------------
               L      W (predicted)
  L (truth): 858    630
  W (truth): 420    1560
-------------------------------------
"AdaBoost": accuracy = 0.5936651583710407
"AdaBoost": precision = 0.6008902077151336
"AdaBoost": recall = 0.6923076923076923
"AdaBoost": f1 = 0.6433677521842733
-------------------------------------
Confusion Matrix:
-------------------------------------
               L      W (predicted)
  L (truth): 251    269
  W (truth): 180    405
-------------------------------------


In [25]:
run_random_forest()

Experiment: train and predict ...
"Random Forest (train)": accuracy = 0.998558246828143
"Random Forest (train)": precision = 1.0
"Random Forest (train)": recall = 0.9974747474747475
"Random Forest (train)": f1 = 0.9987357774968395
-------------------------------------
Confusion Matrix:
-------------------------------------
               L      W (predicted)
  L (truth): 1488    0
  W (truth): 5    1975
-------------------------------------
"Random Forest": accuracy = 0.6153846153846154
"Random Forest": precision = 0.6123595505617978
"Random Forest": recall = 0.7452991452991453
"Random Forest": f1 = 0.6723207401696223
-------------------------------------
Confusion Matrix:
-------------------------------------
               L      W (predicted)
  L (truth): 244    276
  W (truth): 149    436
-------------------------------------


In [26]:
run_qda()

Experiment: train and predict ...


/usr/local/lib/python3.10/dist-packages/sklearn/discriminant_analysis.py:926: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


"Quadratic Discriminant Analysis (train)": accuracy = 0.5717993079584776
"Quadratic Discriminant Analysis (train)": precision = 0.5714285714285714
"Quadratic Discriminant Analysis (train)": recall = 1.0
"Quadratic Discriminant Analysis (train)": f1 = 0.7272727272727273
-------------------------------------
Confusion Matrix:
-------------------------------------
               L      W (predicted)
  L (truth): 3    1485
  W (truth): 0    1980
-------------------------------------
"Quadratic Discriminant Analysis": accuracy = 0.5294117647058824
"Quadratic Discriminant Analysis": precision = 0.5294117647058824
"Quadratic Discriminant Analysis": recall = 1.0
"Quadratic Discriminant Analysis": f1 = 0.6923076923076924
-------------------------------------
Confusion Matrix:
-------------------------------------
               L      W (predicted)
  L (truth): 0    520
  W (truth): 0    585
-------------------------------------


In [27]:
run_nn()

Epoch 1/15
87/87 [==============================] - 2s 9ms/step - loss: 43.4443 - accuracy: 0.5090 - val_loss: 3.9139 - val_accuracy: 0.6326
Epoch 2/15
87/87 [==============================] - 0s 5ms/step - loss: 8.8461 - accuracy: 0.5151 - val_loss: 3.3982 - val_accuracy: 0.4841
Epoch 3/15
87/87 [==============================] - 0s 5ms/step - loss: 4.3564 - accuracy: 0.5364 - val_loss: 2.8026 - val_accuracy: 0.6066
Epoch 4/15
87/87 [==============================] - 0s 6ms/step - loss: 3.1919 - accuracy: 0.5256 - val_loss: 2.5659 - val_accuracy: 0.6110
Epoch 5/15
87/87 [==============================] - 0s 5ms/step - loss: 2.7316 - accuracy: 0.5202 - val_loss: 2.4929 - val_accuracy: 0.6225
Epoch 6/15
87/87 [==============================] - 0s 5ms/step - loss: 2.5699 - accuracy: 0.5382 - val_loss: 2.4412 - val_accuracy: 0.6138
Epoch 7/15
87/87 [==============================] - 0s 5ms/step - loss: 2.4927 - accuracy: 0.5429 - val_loss: 2.3922 - val_accuracy: 0.6268
Epoch 8/15
87/87 [=

In [28]:
run_tabnet()

(2774, 198)
(694, 198)
(2774,)
(694,)


/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:75: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 1.02961 | val_0_auc: 0.53042 | val_1_auc: 0.52435 |  0:00:00s
epoch 1  | loss: 0.83235 | val_0_auc: 0.49067 | val_1_auc: 0.45128 |  0:00:01s
epoch 2  | loss: 0.75456 | val_0_auc: 0.5317  | val_1_auc: 0.52249 |  0:00:02s
epoch 3  | loss: 0.7505  | val_0_auc: 0.48917 | val_1_auc: 0.41566 |  0:00:03s
epoch 4  | loss: 0.72513 | val_0_auc: 0.47741 | val_1_auc: 0.43415 |  0:00:03s
epoch 5  | loss: 0.71408 | val_0_auc: 0.4932  | val_1_auc: 0.51246 |  0:00:04s
epoch 6  | loss: 0.70284 | val_0_auc: 0.47895 | val_1_auc: 0.49232 |  0:00:04s
epoch 7  | loss: 0.69959 | val_0_auc: 0.50893 | val_1_auc: 0.5466  |  0:00:05s
epoch 8  | loss: 0.68852 | val_0_auc: 0.52728 | val_1_auc: 0.51747 |  0:00:05s
epoch 9  | loss: 0.6902  | val_0_auc: 0.52357 | val_1_auc: 0.50361 |  0:00:06s
epoch 10 | loss: 0.68496 | val_0_auc: 0.53305 | val_1_auc: 0.50577 |  0:00:06s
epoch 11 | loss: 0.68378 | val_0_auc: 0.54092 | val_1_auc: 0.54074 |  0:00:07s
epoch 12 | loss: 0.68477 | val_0_auc: 0.54645 | val_

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


"TabNet": accuracy = 0.5375565610859728
"TabNet": precision = 0.5564024390243902
"TabNet": recall = 0.6239316239316239
"TabNet": f1 = 0.5882352941176471
-------------------------------------
Confusion Matrix:
-------------------------------------
               L      W (predicted)
  L (truth): 229    291
  W (truth): 220    365
-------------------------------------


{'metrics': {'accuracy': 0.5375565610859728,
  'precision': 0.5564024390243902,
  'recall': 0.6239316239316239,
  'f1': 0.5882352941176471},
 'pred': array([0, 0, 0, ..., 1, 1, 1]),
 'proba': array([[0.60913104, 0.390869  ],
        [0.5141538 , 0.48584625],
        [0.7215219 , 0.27847806],
        ...,
        [0.05820589, 0.9417941 ],
        [0.00671227, 0.99328774],
        [0.22874215, 0.7712579 ]], dtype=float32)}

In [29]:
run_ensemble(['XGBoost', 'AdaBoost', 'TabNet', 'Linear Regression', 'LightGBM'])

"Ensemble/add": accuracy = 0.6072398190045248
"Ensemble/add": precision = 0.5984354628422425
"Ensemble/add": recall = 0.7846153846153846
"Ensemble/add": f1 = 0.6789940828402367
-------------------------------------
Confusion Matrix:
-------------------------------------
               L      W (predicted)
  L (truth): 212    308
  W (truth): 126    459
-------------------------------------
"Ensemble/mult": accuracy = 0.6054298642533936
"Ensemble/mult": precision = 0.5958815958815958
"Ensemble/mult": recall = 0.7914529914529914
"Ensemble/mult": f1 = 0.6798825256975036
-------------------------------------
Confusion Matrix:
-------------------------------------
               L      W (predicted)
  L (truth): 206    314
  W (truth): 122    463
-------------------------------------
"Ensemble/vote": accuracy = 0.6135746606334842
"Ensemble/vote": precision = 0.6064690026954178
"Ensemble/vote": recall = 0.7692307692307693
"Ensemble/vote": f1 = 0.6782215523737755
-----------------------------

In [30]:
all_results

{'Elo (last)': {'metrics': {'accuracy': 0.6470588235294118,
   'precision': 0.6695652173913044,
   'recall': 0.6581196581196581,
   'f1': 0.6637931034482758}},
 'Model Tree': {'metrics': {'accuracy': 0.616289592760181,
   'precision': 0.6148359486447932,
   'recall': 0.7367521367521368,
   'f1': 0.6702954898911353},
  'pred': array([1, 0, 0, ..., 1, 1, 0]),
  'proba': None},
 'Linear Regression': {'metrics': {'accuracy': 0.632579185520362,
   'precision': 0.6211096075778079,
   'recall': 0.7846153846153846,
   'f1': 0.6933534743202417},
  'pred': array([1, 0, 0, ..., 1, 0, 0]),
  'proba': array([[0.1625906 , 0.8374094 ],
         [0.67236778, 0.32763222],
         [0.56642001, 0.43357999],
         ...,
         [0.38313109, 0.61686891],
         [0.56575039, 0.43424961],
         [0.57913297, 0.42086703]])},
 'XGBoost': {'metrics': {'accuracy': 0.5864253393665159,
   'precision': 0.5924855491329479,
   'recall': 0.7008547008547008,
   'f1': 0.6421299921691463},
  'pred': array([1, 0, 